### Step1. Ready for Tranfer Learning

In [ ]:
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize([64,64]),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomCrop(52), #이미지 일부를 랜덤하게 잘라내어 52*52 사이즈로 변경
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], #입력 데이터 정규화(평균, 표준편차)
                             [0.229, 0.224, 0.225]) #데이터 정규화는 모델을 최적화하며 Local Minimum에 빠지는 것 방지
    ]),
    'val': transforms.Compose([
        transforms.Resize([64,64]),
        transforms.RandomCrop(52),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
}

In [ ]:
data_dir = './splitted'

#datasets 준비 + transform
image_datasets = {x: ImageFolder(root=os.path.join(data_dir, x), transform=data_transforms[x])
                  for x in ['train', 'val']}
                  
#미니 배치로 분리하는 역할
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], 
                  batch_size=BATCH_SIZE, shuffle=True, num_workers=4) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

class_names = image_datasets['train'].classes

### Step2. Load Pre-Trained Model

In [ ]:
from torchvision import models

In [ ]:
#1. Pre-Trained 모델 불러오기
resnet = models.resnet50(pretrained=True) 
#pretrained=True: 미리 학습된 모델의 파라미터 값 그대로 가져옴
#          =False: 모델의 구조만 가져오고 파라미터 값 랜덤으로 설정

num_ftrs = resnet.fc.in_features  
resnet.fc = nn.Linear(num_ftrs, 33) #우리 주제에 맞는 채널의 수를 출력하는 layer 추가 
resnet = resnet.to(DEVICE)

criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.Adam(filter(lambda p: p.requires_grad, resnet.parameters()), lr=0.001)
'''
일부 layer의 파라미터만을 업데이트하기 위해
requires_grad = True 로 설정된 layer의 파라미터에만 적용
'''

In [ ]:
from torch.optim import lr_scheduler
#lr_scheduler.StepLR()
#Epoch 따라 lr을 변경하는 역할, 즉 gamma만큼 곱해 lr을 감소

In [ ]:
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

### Step3. Freeze some layers of Pre-Trained Model

In [ ]:
'''
상위 1-5번의 layer는 파라미터 업데이트하지 않도록 고정
하위 6-10번의 layer는 파라미터 학습 과정에서 업데이트하도록
'''
ct = 0
for child in resnet.children(): #resnet.children()은 resnet 모델의 모든 층 정보를 가지고 있음
  ct += 1
  if ct < 6:
    for param in child.parameters():
      param.requires_grad = False

### Step4. Make the function to train and validate model

In [ ]:
def train_resnet(model, criterion, optimizer, scheduler, num_epochs=25):
  best_model_wts = copy.deepcopy(model.state_dict())
  best_acc = 0.0

  for epoch in range(num_epochs):
    print('----------------epoch {}-------------------'.format(epoch+1))
    since = time.time()

    for phase in ['train', 'test']:
      if phase == 'train':
        model.train() #학습 모드 설정
      else:
        model.eval() #평가 모드 설정
      
      running_loss = 0.0
      running_corrects = 0

      for inputs, labels in dataloaders[phase]:
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)

        optimizer.zero_grad() #이전 batch의 gradient가 optimizer에 저장되어 있으니 이를 초기화

        with torch.set_grad_enabled(phase=='train'):
          #torch.set_grad_enabled() 는 모델의 gradient를 업데이트하고 하지 않기 위해 사용
          #학습 단계에서만 모델 gradient 업데이트, 검증 단계에서는 업데이트 x
          outputs = model(inputs)
          _, preds = torch.max(outputs, 1) #가장 높은 값을 가진 것을 예측값으로 저장
          loss = criterion(outputs, labels)

          if phase == 'train':
            loss.backward()  #위 loss를 통해 역전파를 수행하고 계산된 gradient 값을 각 파라미터에 할당
            optimizer.step() #모델의 파라미터 업데이트

        running_loss += loss.item()*inputs.size(0) #모든 데이터 총 loss

          '''
          모든 데이터의 loss를 합산해서 저장하기 위해 하나의 배치에 대해 계산된 loss값에 데이터 수 곱함
          inputs.size(0): DataLoader에서 전달되는 미니 배치의 데이터 수로 배치 사이즈

          '''
        running_corrects += torch.sum(preds==labels.data) #모든 데이터의 예측값 맞았는지 
        
      if phase=='train':
        scheduler.step()


          l_r = [x['lr'] for x in optimizer_ft.param_groups]
          print('learning rate: ', l_r)
        '''
        optimizer_ft.param_groups의 원소는 학습 과정에서의 파라미터를 저장하는 딕셔너리
        스케줄러에 의해 lr 조정 확인
        '''

        epoch_loss = running_loss/dataset_sizes[phase]
        epoch_acc = running_corrects.double()/dataset_sizes[phase]

        print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_oss, epoch_acc))

        if phase=='val' and epoch_acc > best_acc:
          best_acc = epoch_acc
          best_model_wts = copy.deepcopy(model.state_dict())

    time_elapsed = time.time() - since
    print('Completed in {:.0f}m {:.0f}s'.format(time_elapsed//60, time_elapsed%60))
  print('Best val Acc: {:4f}'.format(best_acc))

  model.load_state_dict(best_model_wts)

  return model

### Step5. Execute model training

In [ ]:
model_resnet50 = train_resnet(resnet, criterion, optimizer_ft, 
                              exp_lr_scheduler, num_epochs=EPOCH)
torch.save(model_resnet50, 'resnet50.pt')